In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

from pathlib import Path

import isotools
from isotools.transcriptome import Transcriptome
from isotools.plots import plot_bar, plot_distr, plot_saturation

In [2]:
#Set reference and data paths
reference_folder = Path("/mnt/group/references/genomic/homo_sapiens/sequences")

#Note we are now using the CHM13 Telomere-to-telomere reference
#https://sites.google.com/ucsc.edu/t2tworkinggroup/chm13-cell-line?authuser=0
genome = reference_folder.joinpath('chm13t2t', 'chm13.draft_v1.1.fasta.gz')
anno   = reference_folder.joinpath('chm13t2t', 'chm13.draft_v1.1.gene_annotation.v4.gff3.gz')

In [3]:
#Load reference.

#After the reference is initially imported, can use the pickled file to speed things up
pickled_ref = Path("chmt2t_v1.1.isotools.pkl")

if pickled_ref.exists():
    isoseq = isotools.Transcriptome.from_reference(pickled_ref)
else:
    isoseq = isotools.Transcriptome.from_reference(anno)
    isoseq.save_reference("chmt2t_v1.1.isotools.pkl")


importing reference from .pkl file chmt2t_v1.1.isotools.pkl


In [4]:
sample_dir = Path("/mnt/scratch/isoseq/data/05_t2t_aligned_for_isotools")
sample_dir.exists()

True

In [5]:
stuff = list(sample_dir.glob("**/*.bam"))
stuff

[PosixPath('/mnt/scratch/isoseq/data/05_t2t_aligned_for_isotools/bc1001/aligned.flnc.bam'),
 PosixPath('/mnt/scratch/isoseq/data/05_t2t_aligned_for_isotools/bc1002/aligned.flnc.bam'),
 PosixPath('/mnt/scratch/isoseq/data/05_t2t_aligned_for_isotools/bc1003/aligned.flnc.bam'),
 PosixPath('/mnt/scratch/isoseq/data/05_t2t_aligned_for_isotools/bc1004/aligned.flnc.bam'),
 PosixPath('/mnt/scratch/isoseq/data/05_t2t_aligned_for_isotools/bc1005/aligned.flnc.bam'),
 PosixPath('/mnt/scratch/isoseq/data/05_t2t_aligned_for_isotools/bc1006/aligned.flnc.bam'),
 PosixPath('/mnt/scratch/isoseq/data/05_t2t_aligned_for_isotools/bc1008/aligned.flnc.bam'),
 PosixPath('/mnt/scratch/isoseq/data/05_t2t_aligned_for_isotools/bc1012/aligned.flnc.bam'),
 PosixPath('/mnt/scratch/isoseq/data/05_t2t_aligned_for_isotools/bc1018/aligned.flnc.bam'),
 PosixPath('/mnt/scratch/isoseq/data/05_t2t_aligned_for_isotools/bc1019/aligned.flnc.bam'),
 PosixPath('/mnt/scratch/isoseq/data/05_t2t_aligned_for_isotools/bc1020/aligned.

In [6]:
#find bam files and associate with sample barcodes
sample_bams = pd.DataFrame.from_dict(
    {
        _.parent.stem: _ 
        for _
        in sample_dir.glob("**/*.bam")
    },
    orient = "index",
    columns = ["file"]
).reset_index(
).rename(columns={"index": "barcode"})

sample_bams

,barcode,file
0,bc1001,/mnt/scratch/isoseq/data/05_t2t_aligned_for_is...
1,bc1002,/mnt/scratch/isoseq/data/05_t2t_aligned_for_is...
2,bc1003,/mnt/scratch/isoseq/data/05_t2t_aligned_for_is...
3,bc1004,/mnt/scratch/isoseq/data/05_t2t_aligned_for_is...
4,bc1005,/mnt/scratch/isoseq/data/05_t2t_aligned_for_is...
5,bc1006,/mnt/scratch/isoseq/data/05_t2t_aligned_for_is...
6,bc1008,/mnt/scratch/isoseq/data/05_t2t_aligned_for_is...
7,bc1012,/mnt/scratch/isoseq/data/05_t2t_aligned_for_is...
8,bc1018,/mnt/scratch/isoseq/data/05_t2t_aligned_for_is...
9,bc1019,/mnt/scratch/isoseq/data/05_t2t_aligned_for_is...


In [7]:
#import subject metadata
metadata = pd.read_csv(Path("/mnt/scratch/isoseq/sample_info.csv"))
metadata

,barcode,subject_id,disease_class,ethnicity
0,bc1001,590085-5-4,NEG,EA
1,bc1002,590108-5-4,SLE,EA
2,bc1003,541305,NEG,AA
3,bc1004,541308,POS,AA
4,bc1005,541424,POS,AA
5,bc1006,541561,NEG,EA
6,bc1008,541826,POS,EA
7,bc1012,510099,NEG,AA
8,bc1018,550003-6-2,SLE,AA
9,bc1019,500066-6-2,SLE,AA


In [8]:
#Add bam file locations to sample metadata
sample_data = metadata.merge(sample_bams, on = "barcode")
sample_data

,barcode,subject_id,disease_class,ethnicity,file
0,bc1001,590085-5-4,NEG,EA,/mnt/scratch/isoseq/data/05_t2t_aligned_for_is...
1,bc1002,590108-5-4,SLE,EA,/mnt/scratch/isoseq/data/05_t2t_aligned_for_is...
2,bc1003,541305,NEG,AA,/mnt/scratch/isoseq/data/05_t2t_aligned_for_is...
3,bc1004,541308,POS,AA,/mnt/scratch/isoseq/data/05_t2t_aligned_for_is...
4,bc1005,541424,POS,AA,/mnt/scratch/isoseq/data/05_t2t_aligned_for_is...
5,bc1006,541561,NEG,EA,/mnt/scratch/isoseq/data/05_t2t_aligned_for_is...
6,bc1008,541826,POS,EA,/mnt/scratch/isoseq/data/05_t2t_aligned_for_is...
7,bc1012,510099,NEG,AA,/mnt/scratch/isoseq/data/05_t2t_aligned_for_is...
8,bc1018,550003-6-2,SLE,AA,/mnt/scratch/isoseq/data/05_t2t_aligned_for_is...
9,bc1019,500066-6-2,SLE,AA,/mnt/scratch/isoseq/data/05_t2t_aligned_for_is...


In [ ]:
sample_pickle = Path("all_pilot_samples_unclustered.pkl")
if sample_pickle.exists():
    isoseq.load(sample_pickle)
else:
    for sample in tqdm(sample_data.itertuples()):
        isoseq.add_sample_from_bam(
            sample.file,
            sample_name=sample.subject_id,
            group=f'{sample.ethnicity}_{sample.disease_class}'
        )
    isoseq.save(sample_pickle)

adding sample 590085-5-4 from file /mnt/scratch/isoseq/data/05_t2t_aligned_for_isotools/bc1001/aligned.flnc.bam
100%|##########| 314701.0/314701 [01:23<00:00, 3774.84reads/s, chr=chrM] 
ignored 23 chimeric alignments with only one part aligned to specified chromosomes.
imported 152 chimeric alignments that can be chained to single nonchimeric transcripts (long intron alingment split)
ignoring 1920 chimeric alignments with less than 2 reads
imported 307829 nonchimeric reads (including  152 chained chimeric alignments) and 766 chimeric reads with coverage of at least 2.
adding sample 590108-5-4 from file /mnt/scratch/isoseq/data/05_t2t_aligned_for_isotools/bc1002/aligned.flnc.bam
100%|##########| 291563.0/291563 [01:19<00:00, 3672.78reads/s, chr=chrM] 
ignored 49 chimeric alignments with only one part aligned to specified chromosomes.
imported 181 chimeric alignments that can be chained to single nonchimeric transcripts (long intron alingment split)
ignoring 3168 chimeric alignments with

In [ ]:
sample_groups={
        "NEG": ["590085-5-4", "541305", "541561", "510099"],
        "POS": ["541308", "541424", "541826", "541522"],
        "SLE": ["590108-5-4","550003-6-2","500066-6-2","500028-6-4"],
    }

In [ ]:
isoseq.add_qc_metrics(genome)
isoseq.make_index()
# isoseq.save("all_pilot_samples.pkl")

In [ ]:
from isotools import DEFAULT_GENE_FILTER, DEFAULT_TRANSCRIPT_FILTER,DEFAULT_REF_TRANSCRIPT_FILTER
ref_filter=DEFAULT_REF_TRANSCRIPT_FILTER
#lets define some custom reference filter flags based on the gencode specific transcript information:
ref_filter['HIGH_SUPPORT']='transcript_support_level=="1"'
ref_filter['PROTEIN_CODING']='transcript_type=="protein_coding"'
print(DEFAULT_TRANSCRIPT_FILTER)

gene_filter=DEFAULT_GENE_FILTER
isoseq.add_filter( gene_filter=gene_filter, ref_transcript_filter=ref_filter)

In [ ]:
#compute some summary statistics on technical artifacts.
tr_stats=[
    isoseq.transcript_length_hist(
        groups=sample_groups,
        add_reference=True,
        min_coverage=2,
        tr_filter=dict(remove=['NOVEL_GENE']),
        ref_filter=dict(include=['HIGH_SUPPORT'])
    ),
    isoseq.downstream_a_hist(
        groups=sample_groups,
        tr_filter=dict( remove=['NOVEL_GENE', 'UNSPLICED']),
        ref_filter=dict(remove=['UNSPLICED'])
    ),
    isoseq.downstream_a_hist(
        groups=sample_groups,
        tr_filter=dict(include=['NOVEL_GENE', 'UNSPLICED'])
    ),
    isoseq.direct_repeat_hist(
        groups=sample_groups,
        bins=np.linspace(-.5,10.5,12)
     )
]

In [ ]:
tr_stats[0][0]

In [ ]:
tr_stats.append(
    (
        pd.concat(
            [
                tr_stats[2][0].add_suffix(
                    ' novel unspliced'
                ),tr_stats[1][0].add_suffix(
                    ' known multiexon'
                )
            ],
            axis=1
        ),tr_stats[2][1]
    )
)

In [ ]:
#statistic on the filter flags
f_stats=isoseq.filter_stats(
    groups=sample_groups,
    weight_by_coverage=True,
    min_coverage=1
)

In [ ]:
f_stats[0].index=f_stats[0].index.str.replace('_','\n')

In [ ]:
#QC plot
from isotools.plots import plot_bar, plot_distr

plt.rcParams["figure.figsize"] = (15,15)
plt.rcParams.update({'font.size': 14})


fig, axs = plt.subplots(2,2)
#A) transcript length
plot_distr(tr_stats[0][0],smooth=3,ax=axs[0,0],**tr_stats[0][1])
#B) internal priming
plot_distr(tr_stats[4][0],smooth=3,ax=axs[0,1],density=True,fill=True, **tr_stats[4][1])
#C) RTTS
plot_distr(tr_stats[3][0],ax=axs[1,0],density=True,**tr_stats[3][1])
#D) frequency of artifacts
plot_bar(f_stats[0],ax=axs[1,1],drop_categories=['MULTIEXON','NOVEL\nTRANSCRIPT','NOVEL','UNSPLICED','NONCANONICAL\nSPLICING' ],**f_stats[1])

fig.tight_layout()

In [ ]:
ref=[[[12,20],[30,40], [50,60],[70,81]],
     [[11,20],[35,40],         [70,79]],
     [[10,20],[30,40], [50,60],[75,80]]]
novel={'FSM':         [[10,20],[30,40], [50,60],[70,80]],
       "5' fragment": [[33,40], [50,60],[70,80]],
       "3' fragment": [[10,20],[30,40], [50,55]],
       "mono exon"  : [[22,35]],
       "exon skipping"     :  [[10,20], [50,60],[70,80]],
       "intron retention"  :  [[10,40], [50,60],[70,80]],
       "novel combination" :  [[10,20],[35,40], [50,60],[75,80]],
       "novel junction"  :   [[10,20],[30,40], [75,80]],
       "novel exonic TSS"  :  [[26,40], [50,60],[70,80]],
       "novel exonic PAS"  :  [[10,20],[30,40], [50,66]],
       "novel 5' splice site":[[10,24],[30,40], [50,60],[70,80]],
       "novel 3' splice site":[[10,20],[26,40], [50,60],[70,80]],
       "novel exon"  :        [[10,20],[30,40],[43,47], [50,60],[70,80]],
       "novel intronic TSS" : [[43,47],[50,60],[70,80]],
       "novel intronic PAS" : [[10,20],[30,40], [82,90]]}
ref={'transcripts':[{'exons':e, 'transcript_name':f'reference {i+1}'} for i,e in enumerate(ref)]}
transcripts=[{'exons':e, 'transcript_name':n} for n,e in novel.items()]
cat=['FSM','ISM','NIC','NNC','novel gene']

In [ ]:
cnr={}
for g, trid, tr in isoseq.iter_transcripts():
    for anno in tr['annotation'][1]:
        cnr[anno]=min(cnr.get(anno,5),tr['annotation'][0])
        
del cnr['FSM']

altsplice=[
    isoseq.altsplice_stats(
        #groups=isoseq.groups(),
        groups=sample_groups,
        weight_by_coverage=True,
        min_coverage=1,
        tr_filter=dict(remove=['RTTS', 'FRAGMENT', 'INTERNAL_PRIMING'])
    ),
    isoseq.altsplice_stats(
        #groups=isoseq.groups(),
        groups=sample_groups,
        weight_by_coverage=True,
        min_coverage=2,
        tr_filter=dict( remove=['RTTS', 'FRAGMENT', 'INTERNAL_PRIMING'])
    ),
    isoseq.altsplice_stats(
        #groups=isoseq.groups(),
        groups=sample_groups,
        weight_by_coverage=False,
        min_coverage=20,
        tr_filter=dict( remove=['RTTS', 'FRAGMENT', 'INTERNAL_PRIMING'])
    )
]
for i in range(3):
    altsplice[i][0].index=altsplice[i][0].index+[f'\n({cat[cnr[subcat]]})' if subcat in cnr else '' for subcat in altsplice[i][0].index]
    altsplice[i][0].index=altsplice[i][0].index.str.replace('splice ','\nsplice ')

In [ ]:
from isotools.plots import plot_bar, plot_distr

plt.rcParams["figure.figsize"] = (20,10)
_=plot_bar(altsplice[0][0],bar_width=.9,ylabel='fraction of reads [%]', legend=False, rot=90,)

In [ ]:
import re

In [ ]:
coord = re.compile("[0-9]+\-[0-9]+")

In [ ]:
coord

In [ ]:
coord.search(str(isoseq['MAPT']).split(" ")[2]).group(0).split("-")

In [ ]:
isoseq['MAPT'].__str__().split(" ")[2]

In [ ]:
goi = "IKZF1"

In [ ]:
plt.rcParams["figure.figsize"] = (20,15)
fig,axs=isoseq[goi].sashimi_figure(samples=sample_groups, x_range=[int(_) for _ in coord.search(isoseq[goi].region).group(0).split("-")])
fig.tight_layout()

In [ ]:
goi = "IFI44L"
plt.rcParams["figure.figsize"] = (20,15)
fig,axs=isoseq[goi].sashimi_figure(samples=sample_groups, x_range=[int(_) for _ in coord.search(isoseq[goi].region).group(0).split("-")])
fig.tight_layout()

In [ ]:
from isotools.plots import plot_embedding

In [ ]:
pca={}

splice_events=isoseq.alternative_splicing_events()

plt.rcParams["figure.figsize"] = (15,30)
f,axs=plt.subplots(4,2)
for ax,t in zip(axs.flatten(),['all','3AS','5AS','ES','IR','ME', 'TSS', 'PAS']):
    pca[t]=plot_embedding(splice_events, ax=ax, labels=False, groups=isoseq.groups(), splice_types=t)

axs[0,0].legend(fontsize='medium', ncol=4,handleheight=2.4, labelspacing=0.05, bbox_to_anchor=(0, 1.1), loc='lower left')

In [ ]:
plt.rcParams["figure.figsize"] = (15,25)
umap={}
f,axs=plt.subplots(4,2)
for ax,t in zip(axs.flatten(),['all','3AS','5AS','ES','IR','ME', 'TSS', 'PAS']):
    umap[t]=plot_embedding(splice_events, method='UMAP',n_neighbors=6, ax=ax, labels=False, groups=isoseq.groups(), splice_types=t)


axs[0,0].legend(fontsize='medium', ncol=4,handleheight=2.4, labelspacing=0.05, bbox_to_anchor=(0, 1.1), loc='lower left')

In [ ]:
cname=['FSM','ISM','NIC','NNC','novel gene']
cnr={}
for g, trid, tr in isoseq.iter_transcripts():
    for anno in tr['annotation'][1]:
        cnr[anno]=min(cnr.get(anno,5),tr['annotation'][0])

# del cnr['FSM']
altsplice=[
    isoseq.altsplice_stats(
        groups=sample_groups,
        weight_by_coverage=True,
        min_coverage=1,
        tr_filter=dict(remove=['RTTS', 'FRAGMENT', 'INTERNAL_PRIMING'])
    ),
    isoseq.altsplice_stats(
        groups=sample_groups,
        weight_by_coverage=True,
        min_coverage=2,
        tr_filter=dict(remove=['RTTS', 'FRAGMENT', 'INTERNAL_PRIMING'])
    ),
    isoseq.altsplice_stats(
        groups=sample_groups,
        weight_by_coverage=False,
        min_coverage=20,
        tr_filter=dict(remove=['RTTS', 'FRAGMENT', 'INTERNAL_PRIMING'])
    )
]

for i in range(3):
    altsplice[i][0].index=altsplice[i][0].index+[f'\n({cname[cnr[subcat]]})' if subcat in cnr else '' for subcat in altsplice[i][0].index]
    altsplice[i][0].index=altsplice[i][0].index.str.replace('splice ','\nsplice ')

In [ ]:
from isotools.plots import plot_bar, plot_distr

plt.rcParams["figure.figsize"] = (30,40)
fig, axs = plt.subplots( 5)
for i,ax in enumerate(axs):
    cat=altsplice[0][0].index[i*4+2:(i+1)*4+2]
    plot_bar(altsplice[0][0],bar_width=.9,ax=ax,ylabel='fraction of reads [%]', legend=False, rot=0,drop_categories=[c for c in altsplice[0][0].index if c not in cat])
axs[-1].legend(fontsize='large', ncol=7,handleheight=2.4, labelspacing=0.05, bbox_to_anchor=(0, -.2), loc='upper left')
_=axs[0].set_title('Novel Transcript Categories')
plt.savefig(fname="novel_xscript_cats.png")

In [ ]:
def plot_diff_example(row, isoseq, groups):
    #sashimi plot for differential spliced genes
    plt.rcParams["figure.figsize"] = (15,10)
    #select gene object
    g=isoseq[row.gene_id]
    #make the title
    gn=list(groups)
    novel='novel' if row.novel else 'known'
    title=f'{g.name} {row.splice_type} ({novel}) FDR={row.padj:.2e}: {gn[0]}={row[gn[0]+"_PSI"]*100:.1f} vs {gn[1]}={row[gn[1]+"_PSI"]*100:.1f} PSI'
    print(title)
    start=row.start
    end=row.end
    #select the junctions of interest (which will be marked purple in the plot)
    joi=[(start,end)]
    if row.splice_type=='ME' and g.is_annotated:
        try:
            sg=g.ref_segment_graph
            n1=next(n for n in sg if n[1]==start)
            n2=next(n for n in sg if n[0]==end)
            joi={(start,sg[suc].start) for suc in n1.suc.values() if sg[suc].start<end }
            joi.update({(sg[pre].end,end) for pre in n2.pre.values() if sg[pre].end>start })
        except:
            pass
    #draw the plot for the two sample groups from the comparison
    f,axs=g.sashimi_figure(samples=groups , x_range=(start-500, end+500),junctions_of_interest=joi    )
    axs[0].set_title(title)
    fig.tight_layout()

In [ ]:
from isotools.plots import plot_diff_results
#We are mostly interested in differential splicing. Alternative TSS and PAS are ignored for now.
types_of_interest=['ES','ME','5AS','3AS']
#dict to store the results in
res={}

In [ ]:
diff_cmp='NEG/SLE'
groups={k:sample_groups[k] for k in diff_cmp.split('/')}

#takes about 20 min
res[diff_cmp]=isoseq.altsplice_test(groups).sort_values('pvalue').reset_index()
res[diff_cmp].to_csv(f'encode_diff_betabinomial_{"_".join(groups)}.csv', index=False)
#res[diff_cmp]=pd.read_csv(f'encode_diff_betabinomial_{"_".join(groups)}.csv')
sig=res[diff_cmp].padj<.1
print(f'{sum(sig)} differential splice sites in {len(res[diff_cmp].loc[sig,"gene"].unique())} genes for {" vs ".join(groups)}')
res[diff_cmp][res[diff_cmp].splice_type.isin(types_of_interest)].head(10)

In [ ]:
goi = "ANPEP"
plt.rcParams["figure.figsize"] = (20,15)
fig,axs=isoseq[goi].sashimi_figure(samples=sample_groups, x_range=[int(_) for _ in coord.search(isoseq[goi].region).group(0).split("-")])
fig.tight_layout()

In [ ]:
goi = "PECAM1"
plt.rcParams["figure.figsize"] = (20,15)
fig,axs=isoseq[goi].sashimi_figure(samples=sample_groups, x_range=[int(_) for _ in coord.search(isoseq[goi].region).group(0).split("-")])
fig.tight_layout()

In [ ]:
res[diff_cmp].sort_values("padj")

In [ ]:
res[diff_cmp].loc[:,["gene","padj"]].sort_values("padj").drop_duplicates()

In [ ]:
top_goi = np.unique(res[diff_cmp].loc[:,["gene","padj"]].sort_values("padj").drop_duplicates().loc[:,"gene"].values[1:10])
top_goi

In [ ]:
from typing import Dict, List, Optional

def plot_sashimi(goi: str, obj: isotools.transcriptome.Transcriptome, groups: Optional[Dict[str, List[str]]] = None) -> None:
    
    if groups is None:
        groups = obj.groups()
    
    coord = re.compile("[0-9]+\-[0-9]+")
    plt.rcParams["figure.figsize"] = (20,15)
    
    fig, axs = obj[goi].sashimi_figure(
        samples=groups,
        x_range=[
            int(_)
            for _
            in coord.search(obj[goi].region).group(0).split("-")
        ]
    )
    fig.tight_layout()

In [ ]:
plot_sashimi("PML", isoseq)

In [ ]:
from functools import partial

In [ ]:
ps_part = partial(plot_sashimi, obj=isoseq, groups=sample_groups)
list(map(ps_part, top_goi))

In [ ]:
isoseq.save("all_pilot_samples.pkl")

# Compare Differential splicing

In [ ]:
from isotools.plots import plot_diff_results
#We are mostly interested in differential splicing. Alternative TSS and PAS are ignored for now.
types_of_interest=['ES','ME','5AS','3AS']
#dict to store the results in
res={}

In [ ]:
diff_cmp = "NEG/SLE"

#takes about 20 min
res[diff_cmp]=isoseq.altsplice_test(groups).sort_values('pvalue').reset_index()
res[diff_cmp].to_csv(f'encode_diff_betabinomial_{"_".join(groups)}.csv', index=False)
#res[diff_cmp]=pd.read_csv(f'encode_diff_betabinomial_{"_".join(groups)}.csv')
sig=res[diff_cmp].padj<.1
print(f'{sum(sig)} differential splice sites in {len(res[diff_cmp].loc[sig,"gene"].unique())} genes for {" vs ".join(groups)}')
res[diff_cmp][res[diff_cmp].splice_type.isin(types_of_interest)].head(10).iloc[:,1:10]

In [ ]:
res[diff_cmp][res[diff_cmp].splice_type.isin(types_of_interest)].head(10).iloc[:,1:17]

In [ ]:
plot_diff_example(res[diff_cmp].iloc[1], isoseq, groups)

In [ ]:
plot_diff_example(res[diff_cmp].iloc[2], isoseq, groups)

In [ ]:
plot_diff_example(res[diff_cmp].iloc[3], isoseq, groups)

In [ ]:
result_table=res[diff_cmp]
min_support=2,
min_diff=0.1
grid_shape=(5, 5)
splice_types=['ES', 'ME', '5AS', '3AS']

In [ ]:
plt.rcParams["figure.figsize"] = (20,20)
f,axs,plotted=plot_diff_results(res[diff_cmp], min_diff=.1,grid_shape=(7,5),min_support=2, splice_types=['ES', 'ME', '5AS', '3AS'])
f.savefig(f'encode_diff_betabinomial_{"_".join(groups)}.png')
plotted.to_csv(f'encode_diff_betabinomial_plot_{"_".join(groups)}.csv', index=False)

In [ ]:
sig=(res["NEG/SLE"].padj<.01) & (res["NEG/SLE"].splice_type.isin( ["ME", "ES","5AS","3AS","IR"]))
res["NEG/SLE"].loc[sig].head(20)


In [ ]:
plt.rcParams["figure.figsize"] = (10,10)
plt.rcParams.update({'font.size': 12})
f,axs=plt.subplots(2,2)
for nr,(ax,cov_th) in enumerate(zip(axs.flatten(),[1,2,5,10])):
    plot_saturation(isoseq, ax=ax,cov_th=cov_th, title='',x_range=(1e4,2e4,1e4), legend=False, xlabel='number of reads [million]', ylabel='probability')
    ax.set_title('ABCD'[nr],{'fontsize':20}, loc='left', pad=10)
ax.legend(loc='upper left')
f.tight_layout()